# CGIページソースコード (先に特徴ファイルを計算する必要がある)

In [ ]:
# CGI sourceCode

#!/usr/local/anaconda3/bin/python3
import pickle
import numpy as np
import cgi
from PIL import Image
import glob
featurePath =  PATH_TO_YOUR_SAVED_FEATURE

def searchImages(queryIdx = 0, method = 'RGB'):
#   Search similar images similar to the image wid "queryIdx" by "method"
#   this function will return the sorted ARG as a result
#
    # preprocessing
    try:
        if method == 'RGB':
            with open('featureData/RGB.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
               featureHist = pickle.load(f)
        elif method == 'DCNN':
           with open('featureData/DCNN.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
               featureHist = pickle.load(f)
        elif method == 'HSV':
           with open('featureData/HSV.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
               featureHist = pickle.load(f)
        elif method == 'LUV':
            with open('featureData/LUV.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
               featureHist = pickle.load(f)
    except IOError:
        print("Could not read file")

    m = featureHist.shape[0]
    if queryIdx < 0 or queryIdx >= m:
        print('Index overflow')
        sys.exit()

    # Caculating similarity
    queryHist = featureHist[queryIdx, :] # get query image's color histgram
    distMat = (featureHist - queryHist) ** 2
    dist  = np.sum(distMat, axis = 1)
    dist  = dist / np.linalg.norm(dist)

    # Get the similar images in an order of whoses similiarity is from high to low
    sortedIdx = np.argsort(dist)
    score = np.sort(dist)
    return sortedIdx, score

print("Content-type:text/html\r\n\r\n")


imgPath = 'img/'

imgList = []
for filename in glob.glob(imgPath + 'dog/*.jpg'): #assuming gif
    imgList.append(filename)
for filename in glob.glob(imgPath + 'cat/*.jpg'): #assuming gif
    imgList.append(filename)


m = len(imgList)

num_in_row = 10
row = (int)(m /num_in_row)
remain = m % num_in_row

queryIdx = 0
feature = "RGB"

link = cgi.FieldStorage()
if "queryIdx" in link:
        queryIdx = (int)(link["queryIdx"].value)
if "feature" in link:
        feature = link["feature"].value


sortedIdx, score = searchImages(queryIdx, feature)

if len(sortedIdx) != len(imgList):
        print('Need to update feature database')
        sys.exit()

print ('<html>')
print ('<head>')
print ('<title>Image Searching CGI Program</title>')
print ('</head>')

print ('<body>')
#print(sortedIdx, score)
print ('<h3>Click a picture to find the similar images and select the feature you\'d like to use below</h3>')

print('<table><tbody><tr>')
print('<form action="'+ '?queryIdx=%d&feature=%s'%(queryIdx, 'RGB')  + '" METHOD = "POST">')
print('<input type="submit" value="RGB" />')
print('</form>')

print('<form action="'+ '?queryIdx=%d&feature=%s'%(queryIdx, 'LUV')  + '" METHOD = "POST">')
print('<input type="submit" value="LUV" />')
print('</form>')

print('<form action="'+ '?queryIdx=%d&feature=%s'%(queryIdx, 'HSV')  + '" METHOD = "POST">')
print('<input type="submit" value="HSV" />')
print('</form>')

print('<form action="'+ '?queryIdx=%d&feature=%s'%(queryIdx, 'DCNN')  + '" METHOD = "POST">')
print('<input type="submit" value="DCNN(vgg16 fc7)" />')
print('</form>')

print('</tr></tbody></table>')

print('<h2>Query Image</h2>')
#print(queryIdx, imgList[queryIdx])
print('<img src = "'+ imgList[queryIdx] + '" width = "120" height = "90">')

print('<hr>')
print('<h2>Related Images</h2>')
# print(row, remain)
print("<table border='5' cellspacing='1' cellpadding='2'><tbody>")
# LOOP here
for i in range(row):
        print('<tr>')
        row_num = i * 10
        for j in range(num_in_row):
                idx = row_num + j
                print("<td align='CENTER'>")
                print("<a href = '" + "?queryIdx=%d&feature=%s" %( sortedIdx[idx], feature) + "'><img src='" + imgList[sortedIdx[idx]] + "' width='120' height='90'></a>")
                print("<br>["+ str(idx) + "]:"+ str(score[idx]) + "</td>")
        print('</tr>')

if remain != 0:
        print('<tr>')
        idx = num_in_row * row - 1
        for i in range(remain):
                idx = idx + 1
                print("<td align='CENTER'>")
                print("<a href = '" + "?queryIdx=%d&feature=%s" %( sortedIdx[idx], feature) + "'><img src='" + imgList[sortedIdx[idx]] + "' width='120' height='90'></a>")
                print("<br>["+ str(idx) + "]:"+ str(score[idx]) + "</td>")
        print('</tr>')


print('</tbody></table>')
print('</body>')
print('</html>')